## PreProcess

In [ ]:
"""_summary_
"""
import requests
response = requests.get("http://mirror.coggle.club/seg_risky_training_anno.csv")
# 检查请求是否成功
if response.status_code == 200:
    with open("/home/tipriest/data/seg_risky/seg_risky_training_anno.csv", "wb") as file:
        file.write(response.content)
        print("文件下载成功")
else:
    print("文件下载失败")


In [ ]:
# !apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null
# !pip install ultralytics==8.2.0 numpy pandas opencv-python Pillow matplotlib > /dev/null
# !axel -n 12 -a  http://mirror.coggle.club/seg_risky_training_data_01.zip -o ~/data/seg_risky/seg_risky_training_data_01.zip
# !unzip -q /home/tipriest/data/seg_risky/training_data_10.zip -d /home/tipriest/data/seg_risky/


### Import Libs

In [2]:
import os, shutil
import glob
import json
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
"""
这是一个示例模块。

此模块包含与示例功能相关的代码。
"""
BASE_DATA_PATH = "/home/tipriest/data/seg_risky"
# 一共有1020575的数据
training_anno = pd.read_csv(os.path.join(
    BASE_DATA_PATH, "seg_risky_training_anno.csv"))
train_jpgs = []
for i in range(2, 4, 1):
    train_jpgs += [x.replace('/home/tipriest/data/seg_risky/', '')
                    for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{i}/*.jpg'))]
training_anno = training_anno[training_anno['Path'].isin(train_jpgs)]
training_anno['Polygons'] = training_anno['Polygons'].apply(json.loads)


In [ ]:
# df = pd.read_csv(os.path.join(
#     BASE_DATA_PATH, "seg_risky_training_anno.csv"))
# df[['Prefix', 'Suffix']] = df['Path'].str.split('/', expand=True)
# prefix_counts= df['Prefix'].value_counts()
# print(prefix_counts)
# for i in range(16):
#     folder_name = format(i, 'x')
#     folder_path = f"folder_{folder_name}"
#     train_jpgs = [x for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{folder_name}/*.jpg'))]
#     print(i, len(train_jpgs))


## Format dateset

In [5]:
"""
Format Train and Test Set
"""
YOLO_SEG_DATASET_BASE_PATH = os.path.join(BASE_DATA_PATH, "datasetYOLO")
YOLO_SEG_DATASET_TRAIN_PATH = os.path.join(YOLO_SEG_DATASET_BASE_PATH, "train/")
YOLO_SEG_DATASET_VALID_PATH = os.path.join(YOLO_SEG_DATASET_BASE_PATH, "valid/")

if os.path.exists(YOLO_SEG_DATASET_BASE_PATH):
    shutil.rmtree(YOLO_SEG_DATASET_BASE_PATH)
os.makedirs(YOLO_SEG_DATASET_TRAIN_PATH)
os.makedirs(YOLO_SEG_DATASET_VALID_PATH)

def normalize_polygon(polygon, img_width, img_height):
    normalized_polygon = [(min(x, img_width) / img_width, min(y, img_height) / img_height) for x, y in polygon]
    return normalized_polygon

for row in training_anno.iloc[20000:].iterrows():
    shutil.copy(os.path.join(BASE_DATA_PATH, row[1].Path), YOLO_SEG_DATASET_TRAIN_PATH)
    img = cv2.imread(os.path.join(BASE_DATA_PATH, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(YOLO_SEG_DATASET_TRAIN_PATH + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(polygon, img_width, img_height)
            normalized_coords = ' '.join([f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')

In [6]:
# 制作验证集III
for row in training_anno.iloc[:20000].iterrows():
    shutil.copy(os.path.join(
        BASE_DATA_PATH, row[1].Path), YOLO_SEG_DATASET_VALID_PATH)

    img = cv2.imread(os.path.join(BASE_DATA_PATH, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(
        YOLO_SEG_DATASET_VALID_PATH + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(
                polygon, img_width, img_height)
            normalized_coords = ' '.join(
                [f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')


In [6]:
YOLO_SEG_DATASET_BASE_PATH = os.path.join(BASE_DATA_PATH, "datasetYOLO")

In [10]:
with open(os.path.join(os.getcwd(), "cfg/data.yaml"), 'w') as up:
    data_root = os.path.abspath(YOLO_SEG_DATASET_BASE_PATH)
    up.write(f'''
path: {data_root}
train: train
val: valid

names:
    0: alter
''')

## Load Pretrained Model & Train model

In [ ]:
# !wget http://mirror.coggle.club/yolo/Arial.ttf -O /home/tipriest/.cache/torch/hub/checkpoints/Arial.ttf
# !wget http://mirror.coggle.club/yolo/yolov8n-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n.pt
# !wget http://mirror.coggle.club/yolo/yolov8n-seg-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n-seg.pt

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO
 
if __name__ == '__main__':
    model = YOLO(r'D:\yolo\yolov11\ultralytics-main\datasets\yolo11.yaml')
    model.train(data=r'D:\yolo\yolov11\ultralytics-main\datasets\data.yaml',
                cache=False,
                imgsz=640,
                epochs=100,
                single_cls=False,  # 是否是单类别检测
                batch=8,
                close_mosaic=10,
                workers=0,
                device='0',
                optimizer='SGD',
                amp=True,
                project='runs/train',
                name='exp',
                )

In [ ]:
"""
first train the model
"""
from ultralytics import YOLO
import os
model_base_path = "/home/tipriest/.cache/torch/hub/checkpoints"
# yolov8n-seg、yolov8s-seg、yolov8m-seg、yolov8l-seg和yolov8x-seg，模型的大小依次递增
# model = YOLO(os.path.join(model_base_path, "yolo11x.pt"))
model = YOLO(os.path.join(model_base_path, "yolo11x.pt"))
model.train(data=os.path.join(os.getcwd(), "cfg/data.yaml"), 
        cache=True,
        imgsz=600,
        epochs=100,
        single_cls=True,  # 是否是单类别检测
        batch=16,
        close_mosaic=10,
        workers=0,
        device='0',
        optimizer='SGD',
        amp=True,
        project='runs/train',
        name='exp')


### Continue to train

In [1]:
import os
from ultralytics import YOLO
model = YOLO("./runs/train/train2/weights/last.pt")
results = model.train(data=os.path.join(os.getcwd(), "cfg/data.yaml"), epochs=200, imgsz=620)

New https://pypi.org/project/ultralytics/8.3.22 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.11 🚀 Python-3.10.14 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15959MiB)
engine/trainer: task=detect, mode=train, model=./runs/train/train2/weights/last.pt, data=/home/tipriest/Documents/AlgorithmPractice/src/deepLearning/DataWhale/Yolo/cfg/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=620, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fal

train: Scanning /home/tipriest/data/seg_risky/datasetYOLO/train.cache... 108037 images, 0 backgrounds, 25 corrupt: 100%|██████████| 108037/108037 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/20bc184a4ce6fb2b7da8292ea83b2673.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.021]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/20bf87ceeb9141bb1d23725b683e4b98.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.033]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/22bb834ee6967259179b65d875ac4756.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.002]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/242e19afc691c5d9f60532593f605b1b.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.035]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/2484d02fcdda24fc246af4173cbc9318.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.008]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/24ad


val: Scanning /home/tipriest/data/seg_risky/datasetYOLO/valid.cache... 20000 images, 0 backgrounds, 1 corrupt: 100%|██████████| 20000/20000 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/valid/2bec6ff53af2afd38998a727a2afd3cb.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.057]


...

     36/200      15.6G     0.8441     0.6406      1.027         25        640: 100%|██████████| 6751/6751 [35:14<00:00,  3.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 625/625 [02:18<00:00,  4.50it/s]

                   all      19999      29151      0.976      0.931       0.97      0.857



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     37/200      14.6G     0.9246     0.7106      1.082         38        640:   0%|          | 1/6751 [00:00<59:40,  1.89it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 34.00 MiB. GPU 0 has a total capacity of 15.58 GiB of which 47.31 MiB is free. Including non-PyTorch memory, this process has 14.86 GiB memory in use. Of the allocated memory 14.31 GiB is allocated by PyTorch, and 181.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 训练后需要在这里加载最好的模型

In [7]:
"""
训练之后加载模型
"""
import glob
import os
from ultralytics import YOLO

model = YOLO("./runs/train/train2/weights/best.pt")
BASE_DATA_PATH = "/home/tipriest/data/seg_risky"
test_imgs = glob.glob(os.path.join(BASE_DATA_PATH, "test_set_B/*/*"))
# test_imgs = glob.glob(os.path.join(BASE_DATA_PATH, "data_set/train/0/*"))
# print(os.path.join(BASE_DATA_PATH, "data_set/train/0/*"))


### Use Instance Segmentation

In [ ]:
import numpy as np
from tqdm import tqdm
Polygon = []
for path in tqdm(test_imgs[:]):
    # print(path)
    results = model(path, verbose=False)
    result = results[0]
    if result.masks is None:
        Polygon.append([])
    else:
        image_polygons = []
        for box in result.masks.xy:
            # 将掩膜的坐标转换为Numpy数组
            box_coords = np.array(box)
            # 获取外接矩形(bounding box)
            # print(mask_coords)
            # print()
            if 0 == box_coords.size:
                continue
            x_min = round(float(np.min(box_coords[:, 0])), 1)
            x_max = round(float(np.max(box_coords[:, 0])), 1)
            y_min = round(float(np.min(box_coords[:, 1])), 1)
            y_max = round(float(np.max(box_coords[:, 1])), 1)
            bounding_box = [
                [x_min, y_min],
                [x_min, y_max],
                [x_max, y_max],
                [x_max, y_min]
            ]
            image_polygons.append(bounding_box)
            # print(bounding_box)
        Polygon.append(image_polygons)

### Use Object Detection

In [8]:
import numpy as np
from tqdm import tqdm
Polygon = []
for path in tqdm(test_imgs[:]):
    # print(path)
    results = model(path, verbose=False)
    result = results[0]
    if result.boxes is None:
        Polygon.append([])
    else:
        image_polygons = []
        for box in result.boxes.xyxy:
            if 0 == box.size:
                continue
            # 将掩膜的坐标转换为Numpy数组
            box_coords = np.array(box.cpu())
            # 获取外接矩形(bounding box)
            # print(mask_coords)
            # print()
            if 0 == box.size:
                continue
            x_min = round(float(box_coords[0]), 1)
            x_max = round(float(box_coords[2]), 1)
            y_min = round(float(box_coords[1]), 1)
            y_max = round(float(box_coords[3]), 1)
            bounding_box = [
                [x_min, y_min],
                [x_min, y_max],
                [x_max, y_max],
                [x_max, y_min]
            ]
            image_polygons.append(bounding_box)
            # print(bounding_box)
        Polygon.append(image_polygons)

  0%|          | 1/100000 [00:00<10:15:02,  2.71it/s]/tmp/ipykernel_939972/526748799.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  box_coords = np.array(box.cpu())
100%|██████████| 100000/100000 [15:36<00:00, 106.73it/s]


In [9]:
import pandas as pd
submit = pd.DataFrame({
    'Path': [x.split('/')[-1] for x in test_imgs[:]],
    'Polygon': Polygon
})

In [10]:
submit = pd.merge(submit, pd.DataFrame({'Path': [x.split('/')[-1] for x in test_imgs[:]]}), on='Path', how='right')
submit = submit.fillna('[]')
submit.to_csv('track2_submit.csv', index=None)